# Load Spacy Model

In [3]:
import spacy

nlp = spacy.load('en_core_web_lg')

# Prepare data and load word lists

In [187]:
import pandas as pd
comments = pd.read_csv('data/comments_top500.csv')

test_comment = "The venue was great"#comments.iloc[120].body.replace(u'\n', '')

print(test_comment)

The venue was great


In [153]:
def load_lexicon():
    lexicon = []
    with open('data/sentiments/subjectivity.ttf') as file:
        for line in file.readlines():
            # Aufbau des Lexicons: type=strongsubj len=1 word1=hamper pos1=verb stemmed1=y priorpolarity=negative
            word_entry = {}
            for entry in line.split(" "):
                s = entry.split("=")
                if len(s) == 2:
                    word_entry[s[0]] = s[1].replace(u'\n', '')
            lexicon.append(word_entry)

    return pd.DataFrame(lexicon)

lexicon = load_lexicon()

lexicon_doc = nlp(' '.join(lexicon["word1"].drop_duplicates()))
# Get negatives: lexicon[lexicon["priorpolarity"] == "negative"]
# print(lexicon[lexicon["priorpolarity"] == "negative"]["word1"].to_list())

In [50]:
def review_wordlist(input):
    doc = nlp(input)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

In [49]:
def review_sentences(input):
    doc = nlp(input.strip())
    sentences = []
    for sent in doc.sents:
        sentences.append(review_wordlist(sent.text))
        
    return sentences

In [97]:
def most_similar(word, word_list):
    queries = [(w, word.similarity(w)) for w in word_list if w.is_lower == word.is_lower and w.prob >= -15 and w.vector_norm]
    by_similarity = sorted(queries, key=lambda w: w[1], reverse=True)
    return by_similarity[:10]

#print([w.lower_ for w in most_similar(nlp(u'bad')[0])])


In [189]:
# TODO: How to handle words that have multiple polarities (example: food)
def get_polarity(word):
    word_series = lexicon[lexicon["word1"] == word1]
    polarity = word_series['priorpolarity'].drop_duplicates()
    print(polarity)
    return polarity.item()
    


#print(get_polarity('food'))

In [190]:
sentences = review_sentences(test_comment)

for sent in sentences:
    
    result = {}
    for word in nlp(' '.join(sent)):
        x = most_similar(word, lexicon_doc)
        for res in x:
            word1 = res[0].text
            polarity = get_polarity(word1)
            if polarity in result:
                result[polarity]['count'] +=1
                result[polarity]['sim'] +=res[1]
            else:
                result[polarity] = {'count': 1, 'sim': res[1]}
                
    print(sent, [(sentiment, res['sim']/res['count']) for sentiment, res in result.items()])

4284    positive
4285     neutral
Name: priorpolarity, dtype: object


ValueError: can only convert an array of size 1 to a Python scalar